<img src="../DATA/corp_logo.jpg" width='160'>

# Various Tracking API Methods

## Boosting Tracker
Based off Adaboost algorithm(the same underlying algorithm that HAAR Cascade based face detector used), and evaluation occurs across multiple frames.

#### Pros:
- Very well known and studied algorithm.
#### Cons:
- Does not know when the tracking has failed.
- Much better techniques available.

## MIL(Multiple Instance Learning) Tracker
similar to boosting, but consider a neighbrhood of points around the current location to create multiple instances.

#### Pros:
- Good performance and does not drift as much as boosting
#### Cons:
- Failure to track an object may not be reported back.
- Can not recover from full obstruction.

## KCF(Kernelized Correlation Filters) Tracker
Exploits some properties of MIL tracker and the fact that many data points will overlap, leading to more accurate and faster tracking.

#### Pros:
- Better than MIL and Boosting.
- Greate first choice.
#### Cons:
- Can not recover from full obstruction of object.


## TLD(Tracking, Learning, and Detection) Tracker
The tracker follows the object frome frame to frame. The detector localizes all appearances that have been observed so far and corrects the tracker if necessary. The learning estimates detector's errors and update it to avoid these errors in the future.

#### Pros:
- Good at tracking even with obstruction in frames.
- Track well under large changes in scale.
#### Cons:
- Can provide many false positives.

## MedianFlow Tracker
The tracker tracks the object in both forward and backward directions im time and measure the discrepancies between these two trajectories.

#### Pros:
- Very good at reporting failed tracking.
- Works well with predictable motion.
#### Cons:
- Fails under large motion(fast moving objects).


---

<br>

In [1]:
def ask_for_tracker():
    print("Welcome! What Tracker API would you like to use?")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = input("Please select your tracker: ")
    
    if choice == '0':
        tracker = cv2.legacy.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.legacy.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.legacy.TrackerTLD_create()
    if choice == '4':
        # tracker = cv2.TrackerMedianFlow_create()
        tracker = cv2.legacy.TrackerMedianFlow_create()
    return tracker

In [17]:
str(tracker).split()[1]

'cv2.legacy.TrackerBoosting'

## Monocular Camera Setup
### 1. Single Object Tracking

In [3]:
import cv2


tracker = ask_for_tracker()
tracker_name = str(tracker).split()[1]

# Read video
cap = cv2.VideoCapture(cv2.CAP_V4L2 + 1)
# cap = cv2.VideoCapture(cv2.CAP_V4L2)
# cap.open("/dev/v4l/by-id/usb-Sonix_Technology_Co.__Ltd._USB_2.0_Camera-video-index0")


# Read first frame.
ret, frame = cap.read()


# Special function allows us to draw on the very first frame our desired ROI
roi = cv2.selectROI(frame, False)

# Initialize tracker with first frame and bounding box
ret = tracker.init(frame, roi)

while True:
    # Read a new frame
    ret, frame = cap.read()
    
    
    # Update tracker
    success, roi = tracker.update(frame)
    
    # roi variable is a tuple of 4 floats
    # We need each value and we need them as integers
    (x,y,w,h) = tuple(map(int,roi))
    
    # Draw Rectangle as Tracker moves
    if success:
        # Tracking success
        p1 = (x, y)
        p2 = (x+w, y+h)
        rectangle_center = (x+(w / 2), y+(h / 2)) 
        cv2.rectangle(frame, p1, p2, (0,255,0), 3)
    else :
        # Tracking failure
        cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    # Display tracker type on frame
    cv2.putText(frame, tracker_name, (0,430), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2);
    cv2.putText(frame, str(rectangle_center), (0,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2);
    # Display result
    cv2.imshow(tracker_name, frame)

    # Exit if ESC pressed
    if (cv2.waitKey(30) & 0xff) == 27 : 
        break
        
cap.release()
cv2.destroyAllWindows()

Welcome! What Tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


Please select your tracker:  1


### 2. Multi-Object Tracking
For running the cell below, Restart the kernel. then run the imports cell and ask_for_tracker function.

In [1]:
import cv2


# Function for choosing number of different types of tracker APIs
def ask_for_tracker():
    # TODO: Add GOTURN, MOSSE, CSRT
    print("Tracker APIs available: ")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = input("Please select your tracker: ")
    
    return choice


# Function for setting different types of tracker APIs
def create_tracker(choice):
    if choice == '0':
        tracker = cv2.legacy.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.legacy.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.legacy.TrackerTLD_create()
    if choice == '4':
        # tracker = cv2.TrackerMedianFlow_create()
        tracker = cv2.legacy.TrackerMedianFlow_create()
    
    return tracker




# Ask for Tracker 
tracker_number = ask_for_tracker()
#tracker_name = str(tracker).split()[1]


# Create a MultiTracker object
multi_tracker = cv2.legacy.MultiTracker_create()
#multi_tracker = []

# Read video
# cap = cv2.VideoCapture(cv2.CAP_V4L2 + 1)
cap = cv2.VideoCapture()
cap.open("/dev/v4l/by-id/usb-Sonix_Technology_Co.__Ltd._USB_2.0_Camera-video-index0")


# Read first frame.
ret, frame = cap.read()


# Select multiple ROIs in the first frame
# bboxes elements' structure --> (x, y, w, h)
bboxes = cv2.selectROIs("Select ROIs", frame)
cv2.destroyWindow("Select ROIs")

# Initialize trackers for selected ROIs
# It is essential to create new tracker objects in each iteration.
for bbox in bboxes:
    tracker = create_tracker(tracker_number)
    multi_tracker.add(tracker, frame, tuple(bbox))



while True:
    # Read a new frame
    ret, frame = cap.read()

    # Initialize a list for getting center of objects
    bboxes_centers = []
    
    
    # Update the tracker with the current frame
    success, bboxes = multi_tracker.update(frame)

    
    if success:
        # Draw bounding boxes
        # Roi variable is a tuple of 4 floats, we need them as int
        for bbox in bboxes:
            (x, y, w, h) = [int(i) for i in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            bboxes_centers.append((x+(w / 2), y+(h / 2)))
            
    else:
        # Tracking failure
        cv2.putText(frame, "Failure to Detect Tracking!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)


    # Display tracker type on frame
    # cv2.putText(frame, tracker_name, (0,430), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2);
    #cv2.putText(frame, str(rectangle_center), (0,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2);
    
    # Display result
    tracker_name = str(tracker).split()[1]
    cv2.imshow(tracker_name, frame)

    # Exit if ESC pressed
    if (cv2.waitKey(30) & 0xff) == 27: 
        break
        
cap.release()
cv2.destroyAllWindows()

Tracker APIs available: 
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


Please select your tracker:  0


[ WARN:0@1.810] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (2401) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module source reported: Could not read from resource.
[ WARN:0@1.811] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@1.811] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search path

## Stereo Camera Setup

In [2]:
import cv2
import time

# Function for choosing number of different types of tracker APIs
def ask_for_tracker():
    # TODO: Add GOTURN, MOSSE, CSRT
    print("Tracker APIs available: ")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = input("Please select your tracker: ")
    
    return choice


# Function for setting different types of tracker APIs
def create_tracker(choice):
    if choice == '0':
        tracker = cv2.legacy.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.legacy.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.legacy.TrackerTLD_create()
    if choice == '4':
        # tracker = cv2.TrackerMedianFlow_create()
        tracker = cv2.legacy.TrackerMedianFlow_create()
    
    return tracker




# Ask for Tracker 
tracker_number = ask_for_tracker()
#tracker_name = str(tracker).split()[1]


# Create a MultiTracker object
multi_tracker1 = cv2.legacy.MultiTracker_create()
multi_tracker2 = cv2.legacy.MultiTracker_create()
#multi_tracker = []

# Read video
# First Camera
cap1 = cv2.VideoCapture()
cap1.open("/dev/v4l/by-path/pci-0000:00:14.0-usb-0:5:1.0-video-index0")
# cap1 = cv2.VideoCapture(cv2.CAP_V4L2 + 0)

# Read video
# Second Camera
cap2 = cv2.VideoCapture()
cap2.open("/dev/v4l/by-path/pci-0000:00:14.0-usb-0:6:1.0-video-index0")
# cap2 = cv2.VideoCapture(cv2.CAP_V4L2 + 2)


# Read first frame.
ret1, frame1 = cap1.read()

# Read first frame.
ret2, frame2 = cap2.read()

# Select multiple ROIs in the first frame for the first camera
# bboxes elements' structure --> (x, y, w, h)
bboxes1 = cv2.selectROIs("Select ROIs", frame1)
cv2.destroyWindow("Select ROIs")

# Select multiple ROIs in the first frame for the second camera
# bboxes elements' structure --> (x, y, w, h)
bboxes2 = cv2.selectROIs("Select ROIs", frame2)
cv2.destroyWindow("Select ROIs")



# Initialize trackers for selected ROIs(first camera)
# It is essential to create new tracker objects in each iteration.
for bbox in bboxes1:
    tracker = create_tracker(tracker_number)
    multi_tracker1.add(tracker, frame1, tuple(bbox))

# Initialize trackers for selected ROIs(second camera)
# It is essential to create new tracker objects in each iteration.
for bbox in bboxes2:
    tracker = create_tracker(tracker_number)
    multi_tracker2.add(tracker, frame2, tuple(bbox))


# Initialize frame counter for labling circles's centers coordinates
# frame_label1 = 0
# frame_label2 = 0

# Initialize a list for getting center of objects
bboxes_centers1 = []
bboxes_centers2 = []


start_time = time.time()

while True:
    # Read a new frame
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    
    
    # Update the tracker with the current frame
    success1, bboxes1 = multi_tracker1.update(frame1)
    success2, bboxes2 = multi_tracker2.update(frame2)

    # first camera
    if success1:
        # Draw bounding boxes
        # Roi variable is a tuple of 4 floats, we need them as int
        for bbox in bboxes1:
            (x1, y1, w1, h1) = [int(i) for i in bbox]
            cv2.rectangle(frame1, (x1, y1), (x1 + w1, y1 + h1), (0, 255, 0), 2)
            bboxes_centers1.append((x1+(w1 / 2), y1+(h1 / 2)))
            
    else:
        # Tracking failure
        cv2.putText(frame1, "Failure to Detect Tracking!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    
    # second camera    
    if success1:
        # Draw bounding boxes
        # Roi variable is a tuple of 4 floats, we need them as int
        for bbox in bboxes2:
            (x2, y2, w2, h2) = [int(i) for i in bbox]
            cv2.rectangle(frame2, (x2, y2), (x2 + w2, y2 + h2), (0, 255, 0), 2)
            bboxes_centers2.append((x2+(w2 / 2), y2+(h2 / 2)))
            
    else:
        # Tracking failure
        cv2.putText(frame2, "Failure to Detect Tracking!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)


    
    # Display result
    tracker_name = str(tracker).split()[1]
    cv2.imshow(tracker_name + 'camera 1', frame1)
    cv2.imshow(tracker_name + 'camera 2', frame2)
    
    # Exit if ESC pressed
    if (cv2.waitKey(30) & 0xff) == 27: 
        break

cap1.release()
cap2.release()
cv2.destroyAllWindows()

Tracker APIs available: 
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


Please select your tracker:  0


[ WARN:0@242.950] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (2401) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module source reported: Could not read from resource.
[ WARN:0@242.950] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@242.950] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Finish the selection process by pressing ESC button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Finish the selection process by pressing ESC button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


[ WARN:0@243.247] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (2401) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module source reported: Could not read from resource.
[ WARN:0@243.248] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@243.248] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [5]:
import cv2

# Connects to your computer's default camera
# cap = cv2.VideoCapture(0)

# Capture the video from the specific camera connected to your system
cap = cv2.VideoCapture()
cap.open("/dev/v4l/by-path/pci-0000:00:14.0-usb-0:5:1.0-video-index0")


# Automatically grab width and height from video feed
# (returns float which we need to convert to integer for later on!)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

ret, frame = cap.read()

while True:
    
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

# When everything done, release the capture and destroy the windows
cap.release()
cv2.destroyAllWindows()

[ WARN:0@143.698] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (2401) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module source reported: Could not read from resource.
[ WARN:0@143.699] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@143.699] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
